# Implementation of the language models

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text.models import * 
from fastai import *

This module fully implements the [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf) from Stephen Merity et al. The main idea of the article is to use a [RNN](http://www.pnas.org/content/79/8/2554) with dropout everywhere, but in an intelligent way. There is a difference with the usual dropout, which is why you’ll see a [`RNNDropout`](/text.models.html#RNNDropout) module: we zero things, as is usual in dropout, but we always zero the same thing according to the sequence dimension (which is the first dimension in pytorch). This ensures consistency when updating the hidden state through the whole sentences/articles. 

This being given, there are five different dropouts in the AWD-LSTM:
- the first one, embedding dropout, is applied when we look the ids of our tokens inside the embedding matrix (to transform them from numbers to a vector of float). We zero some lines of it, so random ids are sent to a vector of zeros instead of being sent to their embedding vector.
- the second one, input dropout, is applied to the result of the embedding with dropout. We forget random pieces of the embedding matrix (but as stated in the last paragraph, the same ones in the sequence dimension).
- the third one is the weight dropout. It’s the trickiest to implement as we randomly replace by 0s some weights of the hidden-to-hidden matrix inside the RNN: this needs to be done in a way that ensure the gradients are still computed and the initial weights still updated.
- the fourth one is the hidden dropout. It’s applied to the output of one of the layers of the RNN before it’s used as input of the next layer (again same coordinates are zeroed in the sequence dimension). This one isn’t applied to the last output, but rather…
- the fifth one is the output dropout, it’s applied to the last output of the model (and like the others, it’s applied the same way through the first dimension).

## Basic functions to get a model

In [ ]:
show_doc(get_language_model)

<h4 id="get_language_model"><code>get_language_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L206" class="source_link">[source]</a></h4>

> <code>get_language_model</code>(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `tie_weights`:`bool`=`True`, `qrnn`:`bool`=`False`, `bias`:`bool`=`True`, `bidir`:`bool`=`False`, `output_p`:`float`=`0.4`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create a full AWD-LSTM.  

The first embedding of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, RNNs with `n_layers` that can be bidirectional if `bidir` is True. The last RNN as an output size of `emb_sz` so that we can use the same decoder as the encoder if `tie_weights` is True. The decoder is a `Linear` layer with or without `bias`. If `qrnn` is set to True, we use [QRNN cells] instead of LSTMS. `pad_token` is the token used for padding.

`embed_p` is used for the embedding dropout, `input_p` is used for the input dropout, `weight_p` is used for the weight dropout, `hidden_p` is used for the hidden dropout and `output_p` is used for the output dropout.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer)). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer).

In [ ]:
show_doc(get_rnn_classifier)

<h4 id="get_rnn_classifier"><code>get_rnn_classifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L217" class="source_link">[source]</a></h4>

> <code>get_rnn_classifier</code>(`bptt`:`int`, `max_seq`:`int`, `n_class`:`int`, `vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\], `bidir`:`bool`=`False`, `qrnn`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create a RNN classifier model.  

This model uses an encoder taken from an AWD-LSTM with arguments `vocab_sz`, `emb_sz`, `n_hid`, `n_layers`, `bias`, `bidir`, `qrnn`, `pad_token` and the dropouts parameters. This encoder is fed the sequence by successive bits of size `bptt` and we only keep the last `max_seq` outputs for the pooling layers.

The decoder use a concatenation of the last outputs, a `MaxPooling` of all the ouputs and an `AveragePooling` of all the outputs. It then uses a list of `BatchNorm`, `Dropout`, `Linear`, `ReLU` blocks (with no `ReLU` in the last one), using a first layer size of `3*emb_sz` then follwoing the numbers in `n_layers` to stop at `n_class`. The dropouts probabilities are read in `drops`.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer)). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer).

## Basic NLP modules

On top of the pytorch or the fastai [`layers`](/layers.html#layers), the language models use some custom layers specific to NLP.

In [ ]:
show_doc(EmbeddingDropout, title_level=3)

<h3 id="EmbeddingDropout"><code>class</code> <code>EmbeddingDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L54" class="source_link">[source]</a></h3>

> <code>EmbeddingDropout</code>(`emb`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `embed_p`:`float`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Apply dropout with probabily `embed_p` to an embedding layer `emb`.  

Each row of the embedding matrix has a probability `embed_p` of being replaced by zeros while the others are rescaled accordingly. 

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[ 0.5133,  2.3739, -0.2393,  0.0190,  5.9955, -1.1347, -2.0073],
        [-1.6842, -2.3767, -0.6625, -2.8638, -2.1798, -2.9583,  2.9088],
        [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
        [ 0.9257, -0.3395, -4.7028, -2.0088, -2.1644, -0.7499,  0.3336],
        [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
        [ 1.1576, -2.1401, -1.1910, -0.5666,  0.3786, -1.5003, -3.4213],
        [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
        [ 0.5133,  2.3739, -0.2393,  0.0190,  5.9955, -1.1347, -2.0073]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
show_doc(RNNDropout, title_level=3)

<h3 id="RNNDropout"><code>class</code> <code>RNNDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L11" class="source_link">[source]</a></h3>

> <code>RNNDropout</code>(`p`:`float`=`0.5`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Dropout with probability `p` that is consistent on the seq_len dimension.  

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[ 0.9925, -0.2256,  0.3223,  1.1966,  0.9948,  0.0428, -0.1508],
          [-2.1964, -2.4128,  0.2171,  0.9676, -0.7302, -0.1933, -0.7311],
          [-0.4719, -1.8729, -0.0717,  0.1136,  0.7460,  0.8771, -0.7895]],
 
         [[-0.1271,  1.0285,  0.9008, -1.3732,  0.0839, -1.1660, -0.3819],
          [ 0.1187, -0.4720, -0.1045, -0.3798, -0.9119, -0.1456, -1.0454],
          [ 1.8361, -1.0637,  0.2853,  1.9379, -0.2943, -1.4134, -1.1985]],
 
         [[ 0.8120,  0.7239,  0.2517, -0.9598, -1.2584, -0.4002, -0.8644],
          [-0.3155, -1.1882,  0.5829, -0.6463,  0.5759, -0.9514, -0.2117],
          [ 1.9468, -1.2213,  0.5912, -0.4159, -0.4183, -0.2705, -0.6499]]]),
 tensor([[[ 1.4179, -0.3223,  0.4604,  1.7094,  1.4211,  0.0000, -0.0000],
          [-0.0000, -3.4468,  0.0000,  1.3823, -1.0432, -0.0000, -1.0444],
          [-0.6741, -2.6755, -0.1025,  0.0000,  1.0657,  1.2530, -0.0000]],
 
         [[-0.1815,  1.4693,  1.2868, -1.9617,  0.1198, -0.0000, -0.0000],
          [ 0

In [ ]:
show_doc(WeightDropout, title_level=3)

<h3 id="WeightDropout"><code>class</code> <code>WeightDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L23" class="source_link">[source]</a></h3>

> <code>WeightDropout</code>(`module`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `weight_p`:`float`, `layer_names`:`StrList`=`['weight_hh_l0']`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

A module that warps another layer in which some weights will be replaced by 0 during training.  

Applies dropout of probability `weight_p` to the layers in `layer_names` of `module` in training mode. A copy of those weights is kept so that the dropout mask can change at every batch.

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[ 0.3355, -0.0212],
        [ 0.7019, -0.4262],
        [-0.6695, -0.6837],
        [-0.5267, -0.7050],
        [-0.5731,  0.0049],
        [-0.4302, -0.3659],
        [ 0.6213, -0.5902],
        [ 0.2209, -0.3167]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[ 0.5591, -0.0354],
        [ 1.1698, -0.0000],
        [-1.1158, -0.0000],
        [-0.8779, -1.1749],
        [-0.0000,  0.0082],
        [-0.7171, -0.6099],
        [ 1.0354, -0.0000],
        [ 0.0000, -0.0000]], grad_fn=<MulBackward0>)

In [ ]:
show_doc(SequentialRNN, title_level=3)

<h3 id="SequentialRNN"><code>class</code> <code>SequentialRNN</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L152" class="source_link">[source]</a></h3>

> <code>SequentialRNN</code>(`args`) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

A sequential module that passes the reset call to its children.  

In [ ]:
show_doc(SequentialRNN.reset)

<h4 id="SequentialRNN.reset"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L154" class="source_link">[source]</a></h4>

> <code>reset</code>()

Call the `reset` function of [`self.children`](/torch_core.html#children) (if they have one).

In [ ]:
show_doc(dropout_mask)

<h4 id="dropout_mask"><code>dropout_mask</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L7" class="source_link">[source]</a></h4>

> <code>dropout_mask</code>(`x`:`Tensor`, `sz`:`Collection`\[`int`\], `p`:`float`)

Return a dropout mask of the same type as `x`, size `sz`, with probability `p` to cancel an element.  

In [ ]:
tst_input = torch.randn(3,3,7)
dropout_mask(tst_input, (3,7), 0.3)

tensor([[0.0000, 0.0000, 1.4286, 1.4286, 0.0000, 1.4286, 1.4286],
        [1.4286, 0.0000, 0.0000, 1.4286, 1.4286, 1.4286, 0.0000],
        [1.4286, 1.4286, 1.4286, 0.0000, 0.0000, 1.4286, 1.4286]])

Such a mask is then expanded in the sequence length dimension and multiplied by the input to do an [`RNNDropout`](/text.models.html#RNNDropout).

## Language model modules

In [ ]:
show_doc(RNNCore, title_level=3)

<h3 id="RNNCore"><code>class</code> <code>RNNCore</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L77" class="source_link">[source]</a></h3>

> <code>RNNCore</code>(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `bidir`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`, `qrnn`:`bool`=`False`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182.  

This is the encoder of the model with an embedding layer of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, `n_layers` layers. `pad_token` is passed to the `Embedding`, if `bidir` is True, the model is bidirectional. If `qrnn` is True, we use QRNN cells instead of LSTMs. Dropouts are `embed_p`, `input_p`, `weight_p` and `hidden_p`.

In [ ]:
show_doc(RNNCore.reset)

<h4 id="RNNCore.reset"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L127" class="source_link">[source]</a></h4>

> <code>reset</code>()

Reset the hidden states.  

In [ ]:
show_doc(LinearDecoder, title_level=3)

<h3 id="LinearDecoder"><code>class</code> <code>LinearDecoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L134" class="source_link">[source]</a></h3>

> <code>LinearDecoder</code>(`n_out`:`int`, `n_hid`:`int`, `output_p`:`float`, `tie_encoder`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=`None`, `bias`:`bool`=`True`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

To go on top of a RNNCore module and create a Language Model.  

Create a the decoder to go on top of an [`RNNCore`](/text.models.html#RNNCore) encoder and create a language model. `n_hid` is the dimension of the last hidden state of the encoder, `n_out` the size of the output. Dropout of `output_p` is applied. If a `tie_encoder` is passed, it will be used for the weights of the linear layer, that will have `bias` or not.

## Classifier modules

In [ ]:
show_doc(MultiBatchRNNCore, title_level=3)

<h3 id="MultiBatchRNNCore"><code>class</code> <code>MultiBatchRNNCore</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L158" class="source_link">[source]</a></h3>

> <code>MultiBatchRNNCore</code>(`bptt`:`int`, `max_seq`:`int`, `args`, `kwargs`) :: [`RNNCore`](/text.models.html#RNNCore)

Create a RNNCore module that can process a full sentence.  

Text is passed by chunks of sequence length `bptt` and only the last `max_seq` outputs are kept for the next layer. `args` and `kwargs` are passed to the [`RNNCore`](/text.models.html#RNNCore).

In [ ]:
show_doc(MultiBatchRNNCore.concat)

<h4 id="MultiBatchRNNCore.concat"><code>concat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L165" class="source_link">[source]</a></h4>

> <code>concat</code>(`arrs`:`Collection`\[`Tensor`\]) → `Tensor`

Concatenate the `arrs` along the batch dimension.  

In [ ]:
show_doc(PoolingLinearClassifier, title_level=3)

<h3 id="PoolingLinearClassifier"><code>class</code> <code>PoolingLinearClassifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L180" class="source_link">[source]</a></h3>

> <code>PoolingLinearClassifier</code>(`layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\]) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create a linear classifier with pooling.  

The last output, `MaxPooling` of all the outputs and `AvgPooling` of all the outputs are concatenated, then blocks of [`bn_drop_lin`](/layers.html#bn_drop_lin) are stacked, according to the values in [`layers`](/layers.html#layers) and `drops`.

In [ ]:
show_doc(PoolingLinearClassifier.pool)

<h4 id="PoolingLinearClassifier.pool"><code>pool</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L191" class="source_link">[source]</a></h4>

> <code>pool</code>(`x`:`Tensor`, `bs`:`int`, `is_max`:`bool`)

Pool the tensor along the seq_len dimension.  

The input tensor `x` (of batch size `bs`) is pooled along the batch dimension. `is_max` decides if we do an `AvgPooling` or a `MaxPooling`.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(WeightDropout.forward)

<h4 id="WeightDropout.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L41" class="source_link">[source]</a></h4>

> <code>forward</code>(`args`:`ArgStar`)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(RNNCore.forward)

<h4 id="RNNCore.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L106" class="source_link">[source]</a></h4>

> <code>forward</code>(`input`:`LongTensor`) → `Tuple`\[`Tensor`, `Tensor`\]

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(EmbeddingDropout.forward)

<h4 id="EmbeddingDropout.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L63" class="source_link">[source]</a></h4>

> <code>forward</code>(`words`:`LongTensor`, `scale`:`Optional`\[`float`\]=`None`) → `Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(RNNDropout.forward)

<h4 id="RNNDropout.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L18" class="source_link">[source]</a></h4>

> <code>forward</code>(`x`:`Tensor`) → `Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(WeightDropout.reset)

<h4 id="WeightDropout.reset"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L48" class="source_link">[source]</a></h4>

> <code>reset</code>()

In [ ]:
show_doc(PoolingLinearClassifier.forward)

<h4 id="PoolingLinearClassifier.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L196" class="source_link">[source]</a></h4>

> <code>forward</code>(`input`:`Tuple`\[`Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(MultiBatchRNNCore.forward)

<h4 id="MultiBatchRNNCore.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L169" class="source_link">[source]</a></h4>

> <code>forward</code>(`input`:`LongTensor`) → `Tuple`\[`Tensor`, `Tensor`\]

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(LinearDecoder.forward)

<h4 id="LinearDecoder.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L146" class="source_link">[source]</a></h4>

> <code>forward</code>(`input`:`Tuple`\[`Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section